In [1]:
import os 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing import image

2024-04-30 20:06:05.984543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 20:06:05.984659: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 20:06:06.258126: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load the Dataset

In [2]:
train = pd.read_csv('/kaggle/input/planttraits2024/train.csv')
test = pd.read_csv('/kaggle/input/planttraits2024/test.csv')

In [3]:
train.head()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,X18_mean,X26_mean,X50_mean,X3112_mean,X4_sd,X11_sd,X18_sd,X26_sd,X50_sd,X3112_sd
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,0.117484,1.243779,1.849375,50.216034,0.008921,1.601473,0.025441,0.153608,0.279610,15.045054
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.389315,0.642940,1.353468,574.098472,0.003102,0.258078,0.000866,0.034630,0.010165,11.004477
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,8.552908,0.395241,2.343153,1130.096731,NaN,NaN,NaN,NaN,NaN,NaN
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,1.083629,0.154200,1.155308,1042.686546,0.011692,2.818356,0.110673,0.011334,0.229224,141.857187
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,0.657585,10.919966,2.246226,2386.467180,0.006157,1.128000,0.026996,0.553815,0.107092,87.146899


In [4]:
test.head()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m03,VOD_X_1997_2018_multiyear_mean_m04,VOD_X_1997_2018_multiyear_mean_m05,VOD_X_1997_2018_multiyear_mean_m06,VOD_X_1997_2018_multiyear_mean_m07,VOD_X_1997_2018_multiyear_mean_m08,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12
0,201238668,8.086756,2246.500000,127.321426,20.423418,353.381042,17.535713,80,109,90,...,0.385838,0.389460,0.387536,0.374910,0.363712,0.364623,0.379435,0.388294,0.398887,0.397853
1,202310319,10.844286,495.871429,28.023809,18.738306,786.554382,29.292856,130,155,142,...,0.279171,0.300813,0.332776,0.314386,0.291168,0.280947,0.277623,0.276503,0.271212,0.272672
2,202604412,8.105556,378.328583,39.928570,41.885647,722.071167,34.853809,133,134,139,...,0.392073,0.418509,0.437172,0.453239,0.445230,0.435441,0.432758,0.429839,0.417250,0.394216
3,201353439,7.077679,878.785706,70.428574,37.045235,669.389343,25.150000,103,140,116,...,0.384626,0.313667,0.352311,0.401131,0.513455,0.519174,0.474007,0.469071,0.408466,0.366724
4,195351745,4.790555,2299.366699,150.199997,24.136568,462.887695,22.516666,85,114,98,...,0.515619,0.506952,0.462181,0.397052,0.358262,0.369279,0.414420,0.469002,0.519534,0.521837


## Preprocessing Tabular data

In [5]:
# not worring about '_sd' columns for now
sd_columns = [col for col in train.columns if col.endswith('_sd')]
train = train.drop(columns=sd_columns)

#target columns
mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

#drop any train items with outlier beyond specified percentile
#some testing indicates this filtering is close-to-optimal
for column in mean_columns:
    upper_quantile = train[column].quantile(0.98)  
    train = train[(train[column] < upper_quantile)]

In [6]:
train.head()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12,X4_mean,X11_mean,X18_mean,X26_mean,X50_mean,X3112_mean
0,192027691,12.235703,374.466675,62.524445,72.256844,773.592041,33.277779,125,149,136,...,0.444432,0.472432,0.448272,0.403038,0.401753,11.758108,0.117484,1.243779,1.849375,50.216034
1,195542235,17.270555,90.239998,10.351111,38.220940,859.193298,40.009777,124,144,138,...,0.261273,0.279112,0.294724,0.311158,0.480334,15.748846,0.389315,0.642940,1.353468,574.098472
2,196639184,14.254504,902.071411,49.642857,17.873655,387.977753,22.807142,107,133,119,...,0.444734,0.455066,0.458448,0.455440,0.796917,5.291251,8.552908,0.395241,2.343153,1130.096731
3,195728812,18.680834,1473.933350,163.100006,45.009758,381.053986,20.436666,120,131,125,...,0.318389,0.350208,0.349995,0.348838,0.525236,9.568305,1.083629,0.154200,1.155308,1042.686546
4,195251545,0.673204,530.088867,50.857777,38.230709,1323.526855,45.891998,91,146,120,...,0.565290,0.488996,0.457526,0.448166,0.411821,14.528877,0.657585,10.919966,2.246226,2386.467180


## Load the image model 

In [7]:
image_model = EfficientNetB3(weights='imagenet', include_top=False, pooling='avg')

#input resolution for the ImageNet
image_model_x = 300
image_model_y = 300

43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### Functions to use EfficientNet to extract image features into tabular data

Images are turned into 1280 columns of tabular data!

In [8]:
# Define the function to create a TensorFlow dataset for images
def create_dataset(image_paths, batch_size=128):
    def process_path(file_path):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [image_model_x, image_model_y])    #modify this to fit what your ImageNet model expects
        img = preprocess_input(img)
        return img
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    image_ds = path_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return image_ds

def extract_features_with_dataset(dataset, df):
    features_list = []
    for batch_imgs in dataset:
        print(".", end="")  # Print progress
        features = image_model.predict(batch_imgs, verbose=0)
        features_list.extend(features)
    features_array = np.array(features_list)
    
    # Convert the features array into a DataFrame
    features_df = pd.DataFrame(features_array)
    
    features_df.columns = [f'feature_{i}' for i in range(features_array.shape[1])]
    
    new_df = pd.concat([df.reset_index(drop=True), features_df.reset_index(drop=True)], axis=1)
    
    return new_df

### Extract image data for train

In [9]:
train_image_folder = '/kaggle/input/planttraits2024/train_images'

image_paths = [os.path.join(train_image_folder, f"{img_id}.jpeg") for img_id in train['id']]

# Create the dataset
image_dataset = create_dataset(image_paths)

# Extract features and directly insert them into the DataFrame as separate columns
train = extract_features_with_dataset(image_dataset, train)

print(train.head())

.

I0000 00:00:1714507611.536549      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...............................................................................................................................................................................................................................................................................................................................................................................................          id  WORLDCLIM_BIO1_annual_mean_temperature  \
0  192027691                               12.235703   
1  195542235                               17.270555   
2  196639184                               14.254504   
3  195728812                               18.680834   
4  195251545                                0.673204   

   WORLDCLIM_BIO12_annual_precipitation  \
0                            374.466675   
1                             90.239998   
2                            902.071411   
3                           1473.933350   
4                            530.088867   

   WORLDCLIM_BIO13.BI

### Cross-Validation of the train set

In [10]:
#do cross-validation testing (this is relatively slow)
do_cv = True

X_full = train.drop(columns=mean_columns)
Y_full = train[mean_columns]

models = {}

for column in Y_full.columns:

    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.05, max_depth=10)

    if do_cv:
        print(f"\nDoing cross-validation scoring for {column}...")
        scores = cross_val_score(model, X_full, Y_full[column],
                                 cv=KFold(n_splits=3, shuffle=True, random_state=42),
                                 scoring='r2')        
        print(f"R^2 score for {column}: {np.mean(scores)}")
    
    #train model with all data
    print(f"Training model for {column}...")
    model.fit(X_full, Y_full[column])
    models[column] = model


Doing cross-validation scoring for X4_mean...
R^2 score for X4_mean: 0.26983550025714204
Training model for X4_mean...

Doing cross-validation scoring for X11_mean...
R^2 score for X11_mean: 0.27831439157700427
Training model for X11_mean...

Doing cross-validation scoring for X18_mean...
R^2 score for X18_mean: 0.440808116675459
Training model for X18_mean...

Doing cross-validation scoring for X50_mean...
R^2 score for X50_mean: 0.17875918586273865
Training model for X50_mean...

Doing cross-validation scoring for X26_mean...
R^2 score for X26_mean: 0.20103232852821054
Training model for X26_mean...

Doing cross-validation scoring for X3112_mean...
R^2 score for X3112_mean: 0.3135493220930153
Training model for X3112_mean...


### Dealing with null values

In [11]:
mean_values = Y_full.mean()
submission = pd.DataFrame({'id': test['id']})
submission[Y_full.columns] = mean_values

#rename from _mean
submission.columns = submission.columns.str.replace('_mean', '')
submission.head()

/tmp/ipykernel_25/901832651.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  submission[Y_full.columns] = mean_values
/tmp/ipykernel_25/901832651.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  submission[Y_full.columns] = mean_values
/tmp/ipykernel_25/901832651.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  submission[Y_full.columns] = mean_values
/tmp/ipykernel_25/901832651.py:3: FutureWarning: Series.__getitem__ treat

,id,X4,X11,X18,X50,X26,X3112
0,201238668,0.51282,15.790058,2.553687,1.578751,19.297984,1481.69107
1,202310319,0.51282,15.790058,2.553687,1.578751,19.297984,1481.69107
2,202604412,0.51282,15.790058,2.553687,1.578751,19.297984,1481.69107
3,201353439,0.51282,15.790058,2.553687,1.578751,19.297984,1481.69107
4,195351745,0.51282,15.790058,2.553687,1.578751,19.297984,1481.69107


### Extracting Test images

In [12]:
test_image_folder = '/kaggle/input/planttraits2024/test_images'

image_paths = [os.path.join(test_image_folder, f"{img_id}.jpeg") for img_id in test['id']]

# Create the dataset
image_dataset = create_dataset(image_paths)

# Extract features and directly insert them into the DataFrame as separate columns
test = extract_features_with_dataset(image_dataset, test)

test.head()

....................................................

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,feature_1526,feature_1527,feature_1528,feature_1529,feature_1530,feature_1531,feature_1532,feature_1533,feature_1534,feature_1535
0,201238668,8.086756,2246.500000,127.321426,20.423418,353.381042,17.535713,80,109,90,...,-0.045957,-0.105488,-0.210390,-0.230769,0.201752,-0.161783,1.148626,0.351571,-0.189009,0.038845
1,202310319,10.844286,495.871429,28.023809,18.738306,786.554382,29.292856,130,155,142,...,-0.146156,1.381575,-0.112924,0.279223,0.183518,0.288151,0.024350,0.257875,-0.102926,-0.077307
2,202604412,8.105556,378.328583,39.928570,41.885647,722.071167,34.853809,133,134,139,...,0.224093,-0.108055,-0.145434,0.723381,-0.088328,-0.016017,0.189308,0.320566,0.072305,0.415609
3,201353439,7.077679,878.785706,70.428574,37.045235,669.389343,25.150000,103,140,116,...,0.277165,0.523270,0.212417,-0.179983,0.967067,-0.040615,0.491361,-0.003460,-0.101113,0.910882
4,195351745,4.790555,2299.366699,150.199997,24.136568,462.887695,22.516666,85,114,98,...,-0.148776,0.064549,-0.119701,-0.199917,-0.206088,-0.119617,-0.205877,0.341527,-0.107541,-0.101997


### Predictions on Test data

In [13]:
submission['X4'] = models['X4_mean'].predict(test)
submission['X11'] = models['X11_mean'].predict(test)
submission['X18'] = models['X18_mean'].predict(test)
submission['X50'] = models['X50_mean'].predict(test)
submission['X26'] = models['X26_mean'].predict(test)
submission['X3112'] = models['X3112_mean'].predict(test)

submission.head()

,id,X4,X11,X18,X50,X26,X3112
0,201238668,0.576703,11.778187,2.413112,1.411179,12.808224,401.344086
1,202310319,0.431599,18.352812,1.095830,1.605178,8.522114,1814.722046
2,202604412,0.508107,15.115910,1.617166,1.583971,15.959418,1221.607788
3,201353439,0.473149,22.367531,0.271673,1.184170,2.382899,1601.340332
4,195351745,0.497721,11.749450,0.308236,1.585951,2.024112,363.492523


### Saving the Submission

In [14]:
submission.to_csv('submission.csv', index=False)